In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

In [ ]:
file_path = '/content/input_variables.xlsx'
df = pd.read_excel(file_path)

In [ ]:
import pandas as pd


# Filter the DataFrame to exclude rows containing =-inf values
df_filtered = df[~df.astype(str).apply(lambda row: any("=-inf" in cell for cell in row), axis=1)]

# Write the filtered DataFrame back to a new Excel file
df_filtered.to_excel("filtered_file.xlsx", index=False)




In [ ]:
import pandas as pd

# Read the Excel file into a DataFrame
df = pd.read_excel("/content/filtered_file.xlsx")

# Remove rows with blank values
df_filtered = df.dropna()

# Write the filtered DataFrame back to a new Excel file
df_filtered.to_excel("input.xlsx", index=False)


In [ ]:

X = df_filtered.iloc[:, :5].values
y = df_filtered.iloc[:, 5:].values


In [ ]:
from sklearn.preprocessing import RobustScaler

# Define your data preprocessing pipeline
scaler = RobustScaler()
X = scaler.fit_transform(X)


In [ ]:
X.shape

(9507, 5)

In [ ]:
y.shape


(9507, 22)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

# Normalize the data (optional but often recommended for neural networks)


# Split the data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
!pip install tensorflow-addons


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 12.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, LeakyReLU
from keras.optimizers import Nadam

# Define your model class
class MyModel(Sequential):
    def __init__(self):
        super(MyModel, self).__init__()
        self.add(LSTM(8, activation=LeakyReLU(alpha=0.01), return_sequences=True, trainable=False))
        self.add(LSTM(8, activation=LeakyReLU(alpha=0.01), trainable=True))
        self.add(Dense(22))

# Instantiate your model
model = MyModel()

# Reshape the input data to include a timestep dimension
X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_val_reshaped = X_val.reshape(X_val.shape[0], 1, X_val.shape[1])
X_test_reshaped = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Load weights after calling the model
model.build(input_shape=(None, 1, X_train.shape[1]))  # Build the model with input shape
model.load_weights('/content/lstm_model_weights (4).h5', by_name=True)

# Set the second layer to be trainable
model.layers[1].trainable = True

# Define the optimizer with the desired learning rate
optimizer = Nadam(learning_rate=0.002)

# Compile the model with the specified optimizer and loss function
model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber())

# Train the model
history = model.fit(X_train_reshaped, y_train, validation_data=(X_val_reshaped, y_val), epochs=300, batch_size=50, verbose=1)

# Evaluate the model on the test set
y_train_pred = model.predict(X_train_reshaped)
y_val_pred = model.predict(X_val_reshaped)
y_test_pred = model.predict(X_test_reshaped)





Epoch 1/300
58/58 [==============================] - 6s 21ms/step - loss: 4.1220 - val_loss: 3.6785
Epoch 2/300
58/58 [==============================] - 0s 8ms/step - loss: 2.5966 - val_loss: 1.0870
Epoch 3/300
58/58 [==============================] - 0s 7ms/step - loss: 0.6716 - val_loss: 0.4782
Epoch 4/300
58/58 [==============================] - 0s 7ms/step - loss: 0.4310 - val_loss: 0.4169
Epoch 5/300
58/58 [==============================] - 0s 6ms/step - loss: 0.4015 - val_loss: 0.4003
Epoch 6/300
58/58 [==============================] - 0s 5ms/step - loss: 0.3826 - val_loss: 0.3755
Epoch 7/300
58/58 [==============================] - 0s 6ms/step - loss: 0.3566 - val_loss: 0.3501
Epoch 8/300
58/58 [==============================] - 0s 6ms/step - loss: 0.3354 - val_loss: 0.3333
Epoch 9/300
58/58 [==============================] - 0s 6ms/step - loss: 0.3217 - val_loss: 0.3215
Epoch 10/300
58/58 [==============================] - 0s 6ms/step - loss: 0.3125 - val_loss: 0.3128
Epoch 11

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous part.

In [ ]:

# Extract weights and biases for all layers
layer_weights = []
layer_biases = []
for layer in model.layers:
  # Exclude the first LSTM layer (not trainable)
  if layer.name != 'lstm':
    # Get weights (already NumPy arrays due to layer.get_weights())
    weights = layer.get_weights()[0]
    # Get biases (might need reshaping)
    biases = layer.get_weights()[1].reshape(-1)  # Reshape if necessary (e.g., Dense layer)
    layer_weights.append(weights)
    layer_biases.append(biases)

# Create DataFrames for weights and biases
weights_dfs = []
biases_dfs = []
for i, (layer_weight, layer_bias) in enumerate(zip(layer_weights, layer_biases)):
  # ... (similar DataFrame creation logic as before, considering layer names and column names)
    # Ensure the DataFrame creation logic is indented within the loop

# Combine and save to Excel (similar to previous code)
final_df = pd.concat([combined_weights_df, combined_biases_df], axis=0)
final_df.to_excel('lstm_weights_and_biases_from_finetuning.xlsx', index=False)


IndentationError: expected an indented block after 'for' statement on line 17 (<ipython-input-40-e8bfdfba6b2f>, line 22)

In [ ]:
import numpy as np
import pandas as pd

# Assuming your model has 8 layers (replace with actual number)
num_layers = 8

# Placeholder weights (replace with actual model weights retrieval)
weights = []
for _ in range(num_layers):  # Generate dummy weights for demonstration
  weights.append(np.random.rand(10, 5))  # Example shape (10 rows, 5 columns)

# Create a list to store weights as DataFrames (one DataFrame per layer)
weights_dfs = []
for i, layer_weights in enumerate(weights):
  # Convert layer weights to NumPy array (already assumed in this example)
  # weights_array = np.array(layer_weights)
  # Create DataFrame for this layer's weights
  layer_df = pd.DataFrame(layer_weights)
  # Add a column for layer name (optional)
  layer_df['Layer'] = i+1  # Add layer number (starting from 1)
  # Append DataFrame to the list
  weights_dfs.append(layer_df)

# Combine all DataFrames into a single DataFrame for the Excel file
# (This is the key modification)
combined_df = pd.concat(weights_dfs, axis=1)

# Ensure there are 8 columns (pad with empty DataFrames if necessary)
if len(combined_df.columns) < num_layers:
  for i in range(num_layers - len(combined_df.columns)):
    empty_df = pd.DataFrame(columns=[f"Layer_{i+len(combined_df.columns)+1}"])
    combined_df = pd.concat([combined_df, empty_df], axis=1)

# Save the weights DataFrame to an Excel file
combined_df.to_excel('lstm_weights_all_layers_single_sheet.xlsx', index=False)


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# Load your trained LSTM model
model = load_model('your_lstm_model.h5')

# Get the weights of the model
weights = model.get_weights()

# Convert the weights to a numpy array
weights_array = np.array(weights)

# Create a DataFrame from the weights array
weights_df = pd.DataFrame(weights_array)

# Save the weights DataFrame to an Excel file
weights_df.to_excel('lstm_weights.xlsx', index=False)


In [ ]:
import numpy as np
import pandas as pd

# Assuming your model has 8 layers (replace with actual number)
num_layers = 8

# Placeholder weights and biases (replace with actual model weights retrieval)
weights = []
biases = []
for _ in range(num_layers):  # Generate dummy weights and biases for demonstration
    weights.append(np.random.rand(10, 5))  # Example weight shape (10 rows, 5 columns)
    biases.append(np.random.rand(5))  # Example bias shape (5 elements)

# Create lists to store weights and biases as DataFrames
weights_dfs = []
biases_dfs = []
for i, (layer_weights, layer_bias) in enumerate(zip(weights, biases)):
    # Create DataFrames for weights and biases
    weights_df = pd.DataFrame(layer_weights)
    biases_df = pd.DataFrame(layer_bias)
    # Add a column for layer name (optional)
    weights_df['Layer'] = f"W_{i+1}"  # Weights with "W_" prefix
    biases_df['Layer'] = f"B_{i+1}"  # Biases with "B_" prefix
    # Append DataFrames to respective lists
    weights_dfs.append(weights_df)
    biases_dfs.append(biases_df)

# Combine all weight DataFrames into a single DataFrame with MultiIndex
combined_weights_df = pd.concat(weights_dfs, axis=1, keys=[f'W_{i+1}' for i in range(num_layers)])

# Combine all bias DataFrames into a single DataFrame with MultiIndex
combined_biases_df = pd.concat(biases_dfs, axis=1, keys=[f'B_{i+1}' for i in range(num_layers)])

# Create the final DataFrame with weights and biases stacked vertically (weights on top)
final_df = pd.concat([combined_weights_df, combined_biases_df], axis=0)

# Save the final DataFrame to an Excel file with two sheets
with pd.ExcelWriter('lstm_weights_and_biases_two_sheets.xlsx') as writer:
    final_df.to_excel(writer, sheet_name='Weights_Biases')


In [ ]:
from sklearn.metrics import r2_score


# Calculate R-squared for training, validation, and testing sets
r2_train = r2_score(y_train, y_train_pred)
r2_val = r2_score(y_val, y_val_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f'R-squared for Training Set: {r2_train:.3f}')
print(f'R-squared for Validation Set: {r2_val:.3f}')
print(f'R-squared for Testing Set: {r2_test:.3f}')



R-squared for Training Set: 0.907
R-squared for Validation Set: 0.900
R-squared for Testing Set: 0.903


In [ ]:
pip install h5py numpy pandas


In [ ]:
import pandas as pd

# Load data from .h5 file
data = pd.read_hdf('/content/lstm_model_weights.h5')

# Save data to Excel
data.to_excel('output.xlsx', index=False)


In [ ]:
import h5py

# Open the HDF5 file
with h5py.File('/content/lstm_model_weights.h5', 'r') as file:
    # Print out all the keys in the HDF5 file
    print("Keys in the HDF5 file:")
    for key in file.keys():
        print(key)


In [ ]:
import pandas as pd
import h5py

# Load data from .h5 file
with h5py.File('/content/lstm_model_weights.h5', 'r') as file:
    # Check if the key exists
    if 'dense_3' in file:
        # Check if the key is a dataset
        if isinstance(file['dense_3'], h5py.Dataset):
            data = file['dense_3'][:]
        else:
            raise TypeError("The key 'dense_3' exists but is not a dataset.")
    else:
        raise KeyError("The key 'dense_3' does not exist in the HDF5 file.")

# Convert to DataFrame
data_df = pd.DataFrame(data)

# Save data to Excel
data_df.to_excel('output.xlsx', index=False)



In [ ]:
import pandas as pd

# Load data from .h5 file
data = pd.read_hdf('/content/lstm_model_weights.h5', key='top_level_model_weights')

# Save data to Excel
data.to_excel('output.xlsx', index=False)


In [ ]:
import h5py
import numpy as np
import pandas as pd

# Replace 'your_h5_file.hdf5' with the actual filename
filename = '/content/lstm_model_weights.h5'

# Open the HDF5 file in read-only mode
with h5py.File(filename, 'r') as f:

    # Get the dataset you want to convert (replace 'your_dataset_name' with the actual name)

    # Convert the dataset to a NumPy array
    data = np.array(filename)

    # Create a pandas DataFrame from the array
    df = pd.DataFrame(data)

    # Replace 'output.csv' with your desired output filename
    df.to_csv('output.csv', index=False)

print("Converted HDF5 file to CSV successfully!")
